<a href="https://colab.research.google.com/github/Only-Mike/ADHD/blob/main/ADHD_Predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#Import necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [5]:
!git clone 'https://github.com/Only-Mike/ADHD.git'

fatal: destination path 'ADHD' already exists and is not an empty directory.


In [6]:
df = pd.read_csv('/content/ADHD/datasets/KKI_phenotypic.csv')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83 entries, 0 to 82
Data columns (total 23 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ScanDir ID       83 non-null     int64  
 1   Site             83 non-null     int64  
 2   Gender           83 non-null     int64  
 3   Age              83 non-null     float64
 4   Handedness       83 non-null     int64  
 5   DX               83 non-null     int64  
 6   Secondary Dx     15 non-null     object 
 7   ADHD Measure     83 non-null     int64  
 8   ADHD Index       83 non-null     int64  
 9   Inattentive      83 non-null     int64  
 10  Hyper/Impulsive  83 non-null     int64  
 11  IQ Measure       83 non-null     int64  
 12  Verbal IQ        83 non-null     int64  
 13  Performance IQ   83 non-null     int64  
 14  Full2 IQ         0 non-null      float64
 15  Full4 IQ         83 non-null     int64  
 16  Med Status       83 non-null     int64  
 17  QC_Rest_1        8

In [8]:
#dropping unrelevant columns
df = df.drop(columns = ['Site', 'ADHD Measure', 'IQ Measure', 'Full2 IQ', 'QC_Rest_1', 'QC_Rest_2', 'QC_Rest_3', 'QC_Rest_4', 'QC_Anatomical_1', 'QC_Anatomical_2', 'DX'])

In [9]:
df.describe() #We have some false values in the following columns: ADHD Index, Inattentive and Hyper/Impulsive

,ScanDir ID,Gender,Age,Handedness,ADHD Index,Inattentive,Hyper/Impulsive,Verbal IQ,Performance IQ,Full4 IQ,Med Status
count,8.300000e+01,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000
mean,3.449295e+06,0.554217,10.243253,0.927711,14.963855,15.265060,15.734940,113.060241,108.686747,110.012048,1.180723
std,2.098556e+06,0.500073,1.346601,0.303762,198.036178,198.098958,198.141738,14.693909,11.995861,11.935287,0.387128
min,1.018959e+06,0.000000,8.020000,0.000000,-999.000000,-999.000000,-999.000000,81.000000,79.000000,85.000000,1.000000
25%,2.005148e+06,0.000000,9.100000,1.000000,42.000000,42.000000,43.000000,104.000000,102.000000,101.500000,1.000000
50%,2.768273e+06,1.000000,10.120000,1.000000,46.000000,48.000000,48.000000,112.000000,108.000000,111.000000,1.000000
75%,3.915209e+06,1.000000,11.095000,1.000000,61.000000,60.000000,59.500000,121.500000,119.000000,119.000000,1.000000
max,9.922944e+06,1.000000,12.990000,2.000000,90.000000,90.000000,90.000000,146.000000,137.000000,134.000000,2.000000


In [10]:
#Round age for fewer unique values and making into integer
df['Age'] = df['Age'].round(decimals = 0)
df['Age'] = df['Age'].astype(int)

In [11]:
#Removes rows with -999 in the following columns
df = df[df['Inattentive'] != -999]
df = df[df['Hyper/Impulsive'] != -999]
df = df[df['ADHD Index'] != -999]

In [12]:
#But as we can see with the min function, it removed the false values of -999
df.min()

<ipython-input-12-86b8d0be8a1f>:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df.min()


ScanDir ID         1018959
Gender                   0
Age                      8
Handedness               0
ADHD Index              40
Inattentive             40
Hyper/Impulsive         41
Verbal IQ               81
Performance IQ          79
Full4 IQ                85
Med Status               1
dtype: int64

In [13]:
df['Secondary Dx '].unique()

array([nan, 'Simple phobia', 'Simple Phobia', 'simple phobias', 'ODD',
       'Simple Phobia ', 'ODD; Phobia', 'Specific phobia', 'Phobia',
       'social and simple phobia '], dtype=object)

In [14]:
#Making none secondary dx into 0 and any secondary dx into 1
df['Secondary Dx '].replace(('Simple phobia', 'Simple Phobia', 'simple phobias', 'ODD', 'Simple Phobia ', 'ODD; Phobia', 'Specific phobia', 'Phobia', 'social and simple phobia '), (1, 1, 1, 1, 1, 1, 1, 1, 1), inplace=True)
df['Secondary Dx '] = df['Secondary Dx '].fillna(0).astype(int)

In [15]:
#Checking that it worked as intended
df['Secondary Dx '].unique()

array([0, 1])

In [16]:
pip install sdv -q

     |████████████████████████████████| 102 kB 3.7 MB/s 
     |████████████████████████████████| 61 kB 343 kB/s 
     |████████████████████████████████| 47 kB 4.0 MB/s 
     |████████████████████████████████| 53 kB 1.7 MB/s 
     |████████████████████████████████| 1.6 MB 44.3 MB/s 
     |████████████████████████████████| 139 kB 61.2 MB/s 
     |████████████████████████████████| 9.4 MB 51.6 MB/s 
     |████████████████████████████████| 295 kB 43.4 MB/s 
     |████████████████████████████████| 965 kB 62.9 MB/s 
     |████████████████████████████████| 662 kB 43.5 MB/s 
     |████████████████████████████████| 280 kB 66.0 MB/s 
     |████████████████████████████████| 15.3 MB 57.3 MB/s 


In [17]:
from sdv.tabular import GaussianCopula
model = GaussianCopula()
model.fit(df)

In [18]:
#Creating the synthetic data
synthetic_data = model.sample(2000)
synthetic_data.head()

,ScanDir ID,Gender,Age,Handedness,Secondary Dx,ADHD Index,Inattentive,Hyper/Impulsive,Verbal IQ,Performance IQ,Full4 IQ,Med Status
0,2510502,1,12,1,0,55,48,55,110,112,91,1
1,4294412,0,11,1,0,60,58,55,117,93,107,1
2,6754542,1,10,1,0,48,44,52,106,99,103,1
3,6322136,0,9,1,1,72,70,65,127,112,107,1
4,5764285,0,9,1,0,44,45,42,108,133,126,2


In [19]:
#Checking the number of unique values in each column to see if something seems odd
synthetic_data.nunique()

ScanDir ID         2000
Gender                2
Age                   6
Handedness            3
Secondary Dx          2
ADHD Index           51
Inattentive          51
Hyper/Impulsive      49
Verbal IQ            66
Performance IQ       59
Full4 IQ             50
Med Status            2
dtype: int64

In [20]:
#Appending the synthetic data to the original df
df = df.append([synthetic_data])

In [21]:
#There is now 2080 rows each with a unique id
df.nunique()

ScanDir ID         2080
Gender                2
Age                   6
Handedness            3
Secondary Dx          2
ADHD Index           51
Inattentive          51
Hyper/Impulsive      50
Verbal IQ            66
Performance IQ       59
Full4 IQ             50
Med Status            2
dtype: int64

# ADHD Index vs Inattentive vs Hyper/Impulsive

In [22]:
corr_matrix = df.corr()
corr_matrix['ADHD Index'].sort_values(ascending = False)

ADHD Index         1.000000
Inattentive        0.855316
Hyper/Impulsive    0.708064
Secondary Dx       0.254103
Med Status         0.066654
Age                0.041071
Handedness         0.010509
Verbal IQ         -0.024410
ScanDir ID        -0.049319
Performance IQ    -0.067825
Gender            -0.153608
Full4 IQ          -0.196228
Name: ADHD Index, dtype: float64

In [23]:
df.corr()

,ScanDir ID,Gender,Age,Handedness,Secondary Dx,ADHD Index,Inattentive,Hyper/Impulsive,Verbal IQ,Performance IQ,Full4 IQ,Med Status
ScanDir ID,1.000000,0.102336,-0.137229,0.076473,0.106319,-0.049319,-0.037359,-0.113353,-0.008862,-0.061399,0.021731,-0.009154
Gender,0.102336,1.000000,0.072997,-0.104662,0.009449,-0.153608,-0.192676,-0.077955,-0.016296,0.049743,-0.105587,-0.059176
Age,-0.137229,0.072997,1.000000,0.078479,0.023213,0.041071,-0.024164,0.096359,-0.032898,-0.199676,-0.201552,-0.119038
Handedness,0.076473,-0.104662,0.078479,1.000000,-0.025555,0.010509,-0.005283,0.052732,-0.012395,-0.061093,0.004177,0.038094
Secondary Dx,0.106319,0.009449,0.023213,-0.025555,1.000000,0.254103,0.263651,0.264440,0.080965,-0.022356,0.066278,0.062514
ADHD Index,-0.049319,-0.153608,0.041071,0.010509,0.254103,1.000000,0.855316,0.708064,-0.024410,-0.067825,-0.196228,0.066654
Inattentive,-0.037359,-0.192676,-0.024164,-0.005283,0.263651,0.855316,1.000000,0.697718,-0.046538,-0.056770,-0.071669,0.091354
Hyper/Impulsive,-0.113353,-0.077955,0.096359,0.052732,0.264440,0.708064,0.697718,1.000000,-0.033269,-0.134787,-0.137267,0.116457
Verbal IQ,-0.008862,-0.016296,-0.032898,-0.012395,0.080965,-0.024410,-0.046538,-0.033269,1.000000,0.175791,0.520053,-0.140018
Performance IQ,-0.061399,0.049743,-0.199676,-0.061093,-0.022356,-0.067825,-0.056770,-0.134787,0.175791,1.000000,0.518743,0.005498


In [24]:
from pandas.plotting import scatter_matrix

attributes = ["Hyper/Impulsive", "ADHD Index", "Inattentive"]
scatter_matrix(df[attributes], figsize = (16,12))
plt.show()

ImportError: ignored

<Figure size 1152x864 with 9 Axes>

#Linear Regression

In [25]:
# Assign X and y for training
X = df[["Inattentive", "Hyper/Impulsive", "Secondary Dx ", "Gender", "Age", "Handedness"]].values
y = df["ADHD Index"]

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 43)

In [27]:
model = LinearRegression()

In [28]:
model.fit(X_train, y_train)

LinearRegression()

In [48]:
X_new = [[75, 90, 1, 1, 20, 2]]
print(model.predict(X_new))


[80.47386285]


In [49]:
print(model.score(X_test, y_test))

0.7532497086991339


# HARD CODING FOR LABELSSS VIGTIGT!!!!

In [31]:
if model.predict(X_new) >= 60:
    print('You most likely have ADHD or ADD')
else: 
    print('You dont have ADHD')


You dont have ADHD


#K-Nearest Neighbors


In [32]:
from sklearn.neighbors import KNeighborsRegressor

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 21)

In [33]:
knn_model = KNeighborsRegressor(n_neighbors=3)

In [34]:
knn_model.fit(X_train, y_train)

KNeighborsRegressor(n_neighbors=3)

In [50]:
knn_X_new = [[75, 90, 1, 1, 20, 2]]
print(knn_model.predict(knn_X_new))


[73.66666667]


In [51]:
print(knn_model.score(X_test, y_test))

0.6958560448176601


In [38]:
if model.predict(X_new) >= 60:
    print('You most likely have ADHD or ADD')
else: 
    print('You dont have ADHD')


You dont have ADHD


# Random Forest Regressor

In [42]:
from sklearn.ensemble import RandomForestRegressor

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 21)

In [43]:
rfg_model = RandomForestRegressor()

In [44]:
rfg_model.fit(X_train, y_train)

RandomForestRegressor()

In [52]:
rfg_X_new = [[75, 90, 1, 1, 20, 2]]
print(rfg_model.predict(rfg_X_new))

[73.56]


In [53]:
print(rfg_model.score(X_test, y_test))

0.714436713406142


In [47]:
if model.predict(X_new) >= 60:
    print('You most likely have ADHD or ADD')
else: 
    print('You dont have ADHD')

You dont have ADHD
